In [80]:
library(tidyverse)

# Custom package
library(rutils)

In [81]:
dirs <- rutils::get_dev_directories(dev_paths_file = "../dev_paths.txt")
projects <- c("TCGA-CESC", "TCGA-UCS", "TCGA-UCEC", "TCGA-OV")
unified_dsets <- c("unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data")
matrisome_list <- paste0(dirs$data_dir, "/matrisome/matrisome_hs_masterlist.tsv")

In [82]:
dset_idx <- 3

In [83]:
p_thresh = 0.05
lfc_thresh = log2(2)
coxph_coeff_thresh = 0.0
mi_thresh = 0.0
consensus_thresh = 0.0
consensus_n = 5

# Helpers

In [84]:
matrisome_df <- rutils::load_matrisome_df(matrisome_list) %>%
    dplyr::select(gene_symbol, division, category)


get_consensus_col <- function(df, n, thresh = 0) {
    consensus_df <- df %>%
        dplyr::select(geneID, contains("mean")) %>%
        dplyr::mutate(consensus = rowSums(.[-1] > thresh) == n)
    return(df %>% dplyr::mutate(consensus = consensus_df$consensus))
}

Parsed with column specification:
cols(
  Division = col_character(),
  Category = col_character(),
  `Gene Symbol` = col_character(),
  `Gene Name` = col_character(),
  Synonyms = col_character(),
  HGNC_IDs = col_double(),
  `HGNC_IDs Links` = col_double(),
  UniProt_IDs = col_character(),
  Refseq_IDs = col_character(),
  Orthology = col_character(),
  Notes = col_character()
)


In [85]:
norm_counts_df <- read_tsv(paste0(dirs$data_dir, "/", unified_dsets[dset_idx], "/", "norm_counts.tsv"))

Parsed with column specification:
cols(
  .default = col_double(),
  geneID = col_character()
)
See spec(...) for full column specifications.


# DGE analysis

In [86]:
DESeq_results_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_DESeq_results.tsv"))
filtered_DESeq_results_df <- DESeq_results_df %>%
    dplyr::filter(abs(log2FoldChange) > lfc_thresh, padj < p_thresh)
filtered_matrisome_DESeq_results_df <- filtered_DESeq_results_df %>%
    dplyr::filter(geneID %in% matrisome_df$gene_symbol)

n_degs <- nrow(filtered_DESeq_results_df)
deg_prop <- nrow(filtered_DESeq_results_df) / nrow(norm_counts_df)
n_degs_up <- nrow(filtered_DESeq_results_df %>% dplyr::filter(log2FoldChange > 0))
n_degs_down <- nrow(filtered_DESeq_results_df %>% dplyr::filter(log2FoldChange < 0))

n_matrisome_degs <- nrow(filtered_matrisome_DESeq_results_df)
matrisome_deg_prop <- nrow(filtered_matrisome_DESeq_results_df) / nrow(matrisome_df)
n_matrisome_degs_up <- nrow(filtered_matrisome_DESeq_results_df %>% dplyr::filter(log2FoldChange > 0))
n_matrisome_degs_down <- nrow(filtered_matrisome_DESeq_results_df %>% dplyr::filter(log2FoldChange < 0))


paste0("DEGs: ", n_degs)
paste0("DEG prop: ", deg_prop)
paste0("Up: ", n_degs_up)
paste0("Down: ", n_degs_down)
paste0("Matrisome DEGs: ", n_matrisome_degs)
paste0("Matrisome DEG prop: ", matrisome_deg_prop)
paste0("Matrisome up: ", n_matrisome_degs_up)
paste0("Matrisome down: ", n_matrisome_degs_down)

Parsed with column specification:
cols(
  geneID = col_character(),
  baseMean = col_double(),
  log2FoldChange = col_double(),
  lfcSE = col_double(),
  stat = col_double(),
  pvalue = col_double(),
  padj = col_double()
)


[1] "DEGs: 8229"

[1] "DEG prop: 0.406530975200079"

[1] "Up: 4862"

[1] "Down: 3367"

[1] "Matrisome DEGs: 618"

[1] "Matrisome DEG prop: 0.601752677702045"

[1] "Matrisome up: 317"

[1] "Matrisome down: 301"

# Survival analysis

In [87]:
coxph_null_scores_df <- read_tsv(paste0(dirs$analysis_dir, "/meta/", "coxph_null_scores.tsv"))
coxph_results_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_coxph_results.tsv"))
filtered_coxph_results_df <- coxph_results_df %>%
    dplyr::filter(gene_pval < p_thresh)

Parsed with column specification:
cols(
  dataset = col_character(),
  lr_test_pval = col_double(),
  wald_test_pval = col_double(),
  score_test_pval = col_double()
)
Parsed with column specification:
cols(
  geneID = col_character(),
  gene_pval = col_double(),
  gene_coeff = col_double()
)


In [88]:
coxph_null_scores_df

dataset,lr_test_pval,wald_test_pval,score_test_pval
<chr>,<dbl>,<dbl>,<dbl>
unified_cervical_data,0.0004814013,0.000049053,1.224173e-06
unified_uterine_data,0.3392807906,0.989893132,3.053979e-01
unified_uterine_endometrial_data,0.0007739010,0.000000000,1.040876e-06


In [89]:
coxph_null_sig <- (coxph_null_scores_df %>% dplyr::filter(dataset == unified_dsets[dset_idx]))$lr_test_pval < p_thresh
n_coxph_sig <- nrow(filtered_coxph_results_df)
prop_coxph_sig <- n_coxph_sig / nrow(matrisome_df)
n_coxph_sig_protective <- nrow(filtered_coxph_results_df %>% dplyr::filter(gene_coeff < 0))
n_coxph_sig_harmful <- nrow(filtered_coxph_results_df %>% dplyr::filter(gene_coeff > 0))


paste0("Sig. null Cox PH: ", coxph_null_sig)
paste0("Sig. Cox PH genes: ", n_coxph_sig)
paste0("Prop. sig. Cox PH genes: ", prop_coxph_sig)
paste0("Sig. Cox PH protective: ", n_coxph_sig_protective)
paste0("Sig. Cox PH harmful: ", n_coxph_sig_harmful)

[1] "Sig. null Cox PH: TRUE"

[1] "Sig. Cox PH genes: 89"

[1] "Prop. sig. Cox PH genes: 0.0866601752677702"

[1] "Sig. Cox PH protective: 11"

[1] "Sig. Cox PH harmful: 78"

# Regression

## Correlation analysis

In [90]:
cor_results_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_cor_results.tsv"))

filtered_cor_results_df <- cor_results_df %>% dplyr::filter(pval < p_thresh)

n_cor <- nrow(filtered_cor_results_df)
n_cor_down <- nrow(filtered_cor_results_df %>% dplyr::filter(cor < 0))
n_cor_up <- nrow(filtered_cor_results_df %>% dplyr::filter(cor > 0))

paste0("Cor. genes: ", n_cor)
paste0("Neg. cor. genes: ", n_cor_down)
paste0("Pos. cor. genes: ", n_cor_up)

Parsed with column specification:
cols(
  geneID = col_character(),
  cor = col_double(),
  pval = col_double(),
  n = col_double()
)


[1] "Cor. genes: 40"

[1] "Neg. cor. genes: 8"

[1] "Pos. cor. genes: 32"

## MI analysis

In [91]:
mi_survival_results_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_MI_survival_results.tsv"))
filtered_mi_survival_results_df <- mi_survival_results_df %>% dplyr::filter(MI_est_median > mi_thresh)

n_mi <- nrow(filtered_mi_survival_results_df)

paste0("MI: ", n_mi)

Parsed with column specification:
cols(
  geneID = col_character(),
  MI_est_median = col_double()
)


[1] "MI: 458"

## Baselines

In [92]:
reg_baselines_df <- read_tsv(paste0(dirs$analysis_dir, "/meta/", "reg_baselines.tsv"))
mae_baseline = (reg_baselines_df %>% filter(dataset == unified_dsets[dset_idx]))$L1
ev_baseline = (reg_baselines_df %>% filter(dataset == unified_dsets[dset_idx]))$explained_variance
paste0("MAE baseline: ", mae_baseline)
paste0("EV baseline: ", ev_baseline)

Parsed with column specification:
cols(
  dataset = col_character(),
  L2 = col_double(),
  L1 = col_double(),
  R2 = col_double(),
  explained_variance = col_double(),
  n = col_double()
)


[1] "MAE baseline: 415.708333333333"

[1] "EV baseline: 0"

In [93]:
reg_baselines_df

dataset,L2,L1,R2,explained_variance,n
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
unified_cervical_data,641687.7,518.3333,0,0,66
unified_uterine_data,620674.9,516.0741,0,0,27
unified_uterine_endometrial_data,386549.9,415.7083,0,0,24


# Classification

## MI analysis

In [94]:
mi_figo_results_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_MI_figo_results.tsv"))
filtered_mi_figo_results_df <- mi_figo_results_df %>% dplyr::filter(MI_est_median > mi_thresh)

n_mi <- nrow(filtered_mi_figo_results_df)

paste0("MI: ", n_mi)

Parsed with column specification:
cols(
  geneID = col_character(),
  MI_est_median = col_double()
)


[1] "MI: 581"

## Anova analysis

In [95]:
welch_anova_results_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_welch_anova_results.tsv"))
filtered_welch_anova_results_df <- welch_anova_results_df %>% dplyr::filter(padj < p_thresh)

n_sig <- nrow(filtered_welch_anova_results_df)

paste0("Sig. ANOVA: ", n_sig)

Parsed with column specification:
cols(
  geneID = col_character(),
  pval = col_double(),
  padj = col_double()
)


[1] "Sig. ANOVA: 13"

## Baselines

In [96]:
cls_baselines_df <- read_tsv(paste0(dirs$analysis_dir, "/meta/", "cls_baselines.tsv"))
cls_baselines_df
f1_macro_majority_baseline <- (cls_baselines_df %>% filter(dataset == unified_dsets[dset_idx]))$f1_macro_majority
f1_macro_MC_baseline <- (cls_baselines_df %>% filter(dataset == unified_dsets[dset_idx]))$f1_macro_MC


paste0("F1 macro (majority guess) baseline: ", f1_macro_majority_baseline)
paste0("F1 macro (Monte Carlo) baseline: ", f1_macro_MC_baseline)

Parsed with column specification:
cols(
  dataset = col_character(),
  f1_macro_majority = col_double(),
  f1_macro_MC = col_double(),
  n = col_double()
)


dataset,f1_macro_majority,f1_macro_MC,n
<chr>,<dbl>,<dbl>,<dbl>
unified_cervical_data,0.1730769,0.2487013,255
unified_uterine_data,0.1406250,0.2443586,46
unified_uterine_endometrial_data,0.1738095,0.2480810,137


[1] "F1 macro (majority guess) baseline: 0.173809523809524"

[1] "F1 macro (Monte Carlo) baseline: 0.24808098628323"

## GBC

In [97]:
f1_gbc_scores_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_gbc_ref_scores.tsv"))
f1_gbc_results_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_gbc_results.tsv"))
f1_gbc_results_df <- get_consensus_col(f1_gbc_results_df, n = consensus_n, thresh = consensus_thresh)
f1_gbc_scores_df

f1_gbc_avg <- mean(f1_gbc_scores_df$ref_score)
f1_gbc_imp <- f1_gbc_avg > f1_macro_MC_baseline
f1_gbc_baseline_pct_imp <- (f1_gbc_avg - f1_macro_MC_baseline) / f1_macro_MC_baseline
n_f1_gbc_consensus_genes <- nrow(f1_gbc_results_df %>% dplyr::filter(consensus == TRUE))

paste0("GBC F1 avg.: ", f1_gbc_avg)
paste0("GBC F1 impr. over baseline: ", f1_gbc_imp)
paste0("GBC F1 pct. impr. over baseline: ", f1_gbc_baseline_pct_imp)
paste0("GBC F1 consensus genes: ", n_f1_gbc_consensus_genes)

Parsed with column specification:
cols(
  model = col_double(),
  ref_score = col_double()
)
Parsed with column specification:
cols(
  geneID = col_character(),
  mean_imp_0 = col_double(),
  score_pct_improvement_0 = col_double(),
  mean_imp_1 = col_double(),
  score_pct_improvement_1 = col_double(),
  mean_imp_2 = col_double(),
  score_pct_improvement_2 = col_double(),
  mean_imp_3 = col_double(),
  score_pct_improvement_3 = col_double(),
  mean_imp_4 = col_double(),
  score_pct_improvement_4 = col_double()
)


model,ref_score
<dbl>,<dbl>
0,0.3055761
1,0.3002717
2,0.2761016
3,0.2815376
4,0.2840617


[1] "GBC F1 avg.: 0.289509729759355"

[1] "GBC F1 impr. over baseline: TRUE"

[1] "GBC F1 pct. impr. over baseline: 0.166996850894599"

[1] "GBC F1 consensus genes: 3"

## LR (L1)

In [98]:
f1_l1_lr_scores_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_l1_lr_ref_scores.tsv"))
f1_l1_lr_results_df <- read_tsv(paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_l1_lr_results.tsv"))
f1_l1_lr_results_df <- get_consensus_col(f1_l1_lr_results_df, n = consensus_n, thresh = consensus_thresh)
f1_l1_lr_scores_df

f1_l1_lr_avg <- mean(f1_l1_lr_scores_df$ref_score)
f1_l1_lr_imp <- f1_l1_lr_avg > f1_macro_MC_baseline
f1_l1_lr_pct_imp <- (f1_l1_lr_avg - f1_macro_MC_baseline) / f1_macro_MC_baseline
n_f1_l1_lr_consensus_genes <- nrow(f1_l1_lr_results_df %>% dplyr::filter(consensus == TRUE))

paste0("LR L1 F1 avg.: ", f1_l1_lr_avg)
paste0("LR L1 F1 impr. over baseline: ", f1_l1_lr_imp)
paste0("LR L1 F1 pct. impr. over baseline: ", f1_l1_lr_pct_imp)
paste0("LR L1 F1 consensus genes: ", n_f1_l1_lr_consensus_genes)

Parsed with column specification:
cols(
  model = col_double(),
  ref_score = col_double()
)
Parsed with column specification:
cols(
  geneID = col_character(),
  mean_imp_0 = col_double(),
  score_pct_improvement_0 = col_double(),
  mean_imp_1 = col_double(),
  score_pct_improvement_1 = col_double(),
  mean_imp_2 = col_double(),
  score_pct_improvement_2 = col_double(),
  mean_imp_3 = col_double(),
  score_pct_improvement_3 = col_double(),
  mean_imp_4 = col_double(),
  score_pct_improvement_4 = col_double()
)


model,ref_score
<dbl>,<dbl>
0,0.3366610
1,0.3789767
2,0.3565851
3,0.3464781
4,0.3514332


[1] "LR L1 F1 avg.: 0.354026811258891"

[1] "LR L1 F1 impr. over baseline: TRUE"

[1] "LR L1 F1 pct. impr. over baseline: 0.427061447001443"

[1] "LR L1 F1 consensus genes: 76"